In [1]:
import pandas as pd
import ssl
import numpy as np
from pathlib import Path
import sqlalchemy
import datetime

# from utils.helpers import alpaca_symbol_data
from utils.prep_data import prep_data
from utils.zero_one import zero_one
from utils.one_large import one_large
from utils.negative import negative
# For creating the DB
database_connection_string = 'sqlite:///SP500.db'
engine = sqlalchemy.create_engine(database_connection_string)

# prevent an ssl failure
ssl._create_default_https_context = ssl._create_unverified_context

payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

# There are 2 tables on the Wikipedia page
# we want the first table
first_table = payload[0]
second_table = payload[1]

df500table = first_table

sp500Tickers = df500table['Symbol'].values.tolist()
df500sectors = df500table.drop(columns={'Security','SEC filings','GICS Sub-Industry', 'Headquarters Location', 'Date first added', 'CIK','Founded'})
df500sectors.set_index('Symbol', inplace=True)
sectors = df500sectors['GICS Sector'].unique()
# print(sp500Tickers)

# 1Y Data

The SPY is 1 for positive and 0 for negative

In [3]:
# Normal

StocksWide = pd.read_sql_table('StocksWide', engine)
SectorDF = prep_data(StocksWide, sectors, df500sectors)
SectorDF = zero_one(SectorDF)
SectorDF

c:\Users\canav\Documents\Fintech\machine_learning_index_prediction\utils\prep_data.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SectorDF[sector] = TempDF.mean(axis=1)


,SPY Open,SPY Close,Industrials,Health Care,Information Technology,Communication Services,Consumer Staples,Consumer Discretionary,Utilities,Financials,Materials,Real Estate,Energy,SPY
timestamp,,,,,,,,,,,,,,
2021-04-14 04:00:00,413.79,415.87,-0.000804,0.000509,-0.004617,-0.004491,-0.001597,-0.002539,0.007733,0.010694,0.009282,-0.004979,0.020342,1.0
2021-04-15 04:00:00,417.29,417.26,0.000453,0.011926,0.004855,0.003417,0.005316,-0.004535,0.011355,-0.001082,0.004324,0.011913,-0.006888,0.0
2021-04-16 04:00:00,416.26,415.21,-0.003498,0.002819,-0.002692,-0.003881,0.001754,0.002421,0.002735,-0.004246,-0.001499,-0.003487,-0.018957,0.0
2021-04-19 04:00:00,413.93,412.17,-0.004149,-0.000356,-0.009452,-0.002727,-0.002429,-0.007847,-0.005995,-0.003404,-0.005562,0.002253,-0.001944,0.0
2021-04-20 04:00:00,411.55,416.07,-0.008910,0.002328,-0.005653,-0.007387,0.009700,-0.015166,0.012631,-0.011951,-0.008709,0.011858,-0.025533,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-07 04:00:00,448.04,447.57,0.005856,0.021049,0.005208,-0.009337,0.008555,0.004695,-0.003217,-0.001480,0.004416,-0.004456,0.002626,0.0
2022-04-08 04:00:00,444.11,439.92,-0.005791,0.005426,-0.007141,0.001863,0.001890,0.004482,0.001318,0.002899,0.002020,0.001696,0.027976,0.0
2022-04-11 04:00:00,443.02,438.29,-0.001636,-0.016449,-0.008222,-0.001345,-0.002333,0.006947,-0.014041,-0.002284,-0.001462,-0.010802,-0.018648,0.0


In [ ]:
SectorDF.to_sql('SectorDF', engine, if_exists='replace')

# 1Y Negative

SPY > 0 then 1

SPY < 0 then -1

In [3]:
StocksWide = pd.read_sql_table('StocksWide', engine)
SectorDFNegative = prep_data(StocksWide, sectors, df500sectors)
SectorDFNegative = one_large(SectorDFNegative)
SectorDFNegative

c:\Users\canav\Documents\Fintech\machine_learning_index_prediction\utils\prep_data.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SectorDF[sector] = TempDF.mean(axis=1)


,SPY Open,SPY Close,Industrials,Health Care,Information Technology,Communication Services,Consumer Staples,Consumer Discretionary,Utilities,Financials,Materials,Real Estate,Energy,SPY
timestamp,,,,,,,,,,,,,,
2021-04-14 04:00:00,413.79,415.87,-0.000804,0.000509,-0.004617,-0.004491,-0.001597,-0.002539,0.007733,0.010694,0.009282,-0.004979,0.020342,1.0
2021-04-15 04:00:00,417.29,417.26,0.000453,0.011926,0.004855,0.003417,0.005316,-0.004535,0.011355,-0.001082,0.004324,0.011913,-0.006888,0.0
2021-04-16 04:00:00,416.26,415.21,-0.003498,0.002819,-0.002692,-0.003881,0.001754,0.002421,0.002735,-0.004246,-0.001499,-0.003487,-0.018957,0.0
2021-04-19 04:00:00,413.93,412.17,-0.004149,-0.000356,-0.009452,-0.002727,-0.002429,-0.007847,-0.005995,-0.003404,-0.005562,0.002253,-0.001944,0.0
2021-04-20 04:00:00,411.55,416.07,-0.008910,0.002328,-0.005653,-0.007387,0.009700,-0.015166,0.012631,-0.011951,-0.008709,0.011858,-0.025533,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-07 04:00:00,448.04,447.57,0.005856,0.021049,0.005208,-0.009337,0.008555,0.004695,-0.003217,-0.001480,0.004416,-0.004456,0.002626,0.0
2022-04-08 04:00:00,444.11,439.92,-0.005791,0.005426,-0.007141,0.001863,0.001890,0.004482,0.001318,0.002899,0.002020,0.001696,0.027976,-1.0
2022-04-11 04:00:00,443.02,438.29,-0.001636,-0.016449,-0.008222,-0.001345,-0.002333,0.006947,-0.014041,-0.002284,-0.001462,-0.010802,-0.018648,-1.0


In [4]:
SectorDFNegative.to_sql('SectorDFNegative', engine, if_exists='replace')

# 1Y Larger Seperation


SPY > .005 then 1

SPY < -.005 then -1

in between 0

In [4]:
# Large
StocksWide = pd.read_sql_table('StocksWide', engine)
SectorDFLarge = prep_data(StocksWide, sectors, df500sectors)
SectorDFLarge = one_large(SectorDFLarge)
SectorDFLarge

c:\Users\canav\Documents\Fintech\machine_learning_index_prediction\utils\prep_data.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SectorDF[sector] = TempDF.mean(axis=1)


,SPY Open,SPY Close,Industrials,Health Care,Information Technology,Communication Services,Consumer Staples,Consumer Discretionary,Utilities,Financials,Materials,Real Estate,Energy,SPY
timestamp,,,,,,,,,,,,,,
2021-04-14 04:00:00,413.79,415.87,-0.000804,0.000509,-0.004617,-0.004491,-0.001597,-0.002539,0.007733,0.010694,0.009282,-0.004979,0.020342,1.0
2021-04-15 04:00:00,417.29,417.26,0.000453,0.011926,0.004855,0.003417,0.005316,-0.004535,0.011355,-0.001082,0.004324,0.011913,-0.006888,0.0
2021-04-16 04:00:00,416.26,415.21,-0.003498,0.002819,-0.002692,-0.003881,0.001754,0.002421,0.002735,-0.004246,-0.001499,-0.003487,-0.018957,0.0
2021-04-19 04:00:00,413.93,412.17,-0.004149,-0.000356,-0.009452,-0.002727,-0.002429,-0.007847,-0.005995,-0.003404,-0.005562,0.002253,-0.001944,0.0
2021-04-20 04:00:00,411.55,416.07,-0.008910,0.002328,-0.005653,-0.007387,0.009700,-0.015166,0.012631,-0.011951,-0.008709,0.011858,-0.025533,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-07 04:00:00,448.04,447.57,0.005856,0.021049,0.005208,-0.009337,0.008555,0.004695,-0.003217,-0.001480,0.004416,-0.004456,0.002626,0.0
2022-04-08 04:00:00,444.11,439.92,-0.005791,0.005426,-0.007141,0.001863,0.001890,0.004482,0.001318,0.002899,0.002020,0.001696,0.027976,-1.0
2022-04-11 04:00:00,443.02,438.29,-0.001636,-0.016449,-0.008222,-0.001345,-0.002333,0.006947,-0.014041,-0.002284,-0.001462,-0.010802,-0.018648,-1.0


In [5]:

SectorDFLarge.to_sql('SectorDFLarge', engine, if_exists='replace')

# 3Y Data

In [12]:
StocksWide3Y = pd.read_sql_table('StocksWide3Y', engine)
SectorDF3Y = prep_data(StocksWide3Y, sectors, df500sectors)
SectorDF3Y = zero_one(SectorDF3Y)
SectorDF3Y.to_sql('SectorDF3Y', engine, if_exists='replace')
SectorDF3Y

c:\Users\canav\Documents\Fintech\machine_learning_index_prediction\utils\prep_data.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SectorDF[sector] = TempDF.mean(axis=1)


,SPY Open,SPY Close,Industrials,Health Care,Information Technology,Communication Services,Consumer Staples,Consumer Discretionary,Utilities,Financials,Materials,Real Estate,Energy,SPY
timestamp,,,,,,,,,,,,,,
2021-04-14 04:00:00,413.79,415.87,-0.000804,0.000509,-0.004617,-0.004491,-0.001597,-0.002539,0.007733,0.010694,0.009282,-0.004979,0.020342,1.0
2021-04-15 04:00:00,417.29,417.26,0.000453,0.011926,0.004855,0.003417,0.005316,-0.004535,0.011355,-0.001082,0.004324,0.011913,-0.006888,0.0
2021-04-16 04:00:00,416.26,415.21,-0.003498,0.002819,-0.002692,-0.003881,0.001754,0.002421,0.002735,-0.004246,-0.001499,-0.003487,-0.018957,0.0
2021-04-19 04:00:00,413.93,412.17,-0.004149,-0.000356,-0.009452,-0.002727,-0.002429,-0.007847,-0.005995,-0.003404,-0.005562,0.002253,-0.001944,0.0
2021-04-20 04:00:00,411.55,416.07,-0.008910,0.002328,-0.005653,-0.007387,0.009700,-0.015166,0.012631,-0.011951,-0.008709,0.011858,-0.025533,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-07 04:00:00,448.04,447.57,0.005856,0.021049,0.005208,-0.009337,0.008555,0.004695,-0.003217,-0.001480,0.004416,-0.004456,0.002626,0.0
2022-04-08 04:00:00,444.11,439.92,-0.005791,0.005426,-0.007141,0.001863,0.001890,0.004482,0.001318,0.002899,0.002020,0.001696,0.027976,0.0
2022-04-11 04:00:00,443.02,438.29,-0.001636,-0.016449,-0.008222,-0.001345,-0.002333,0.006947,-0.014041,-0.002284,-0.001462,-0.010802,-0.018648,0.0


In [17]:
StocksWide3Y = pd.read_sql_table('StocksWide3Y', engine)
SectorDF3Y = prep_data(StocksWide3Y, sectors, df500sectors)
SectorDF3YLarge = one_large(SectorDF3Y)
SectorDF3YLarge['SPY'].value_counts()

c:\Users\canav\Documents\Fintech\machine_learning_index_prediction\utils\prep_data.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SectorDF[sector] = TempDF.mean(axis=1)


 0.0    135
 1.0     66
-1.0     53
Name: SPY, dtype: int64

In [21]:
SectorDF3YLarge.to_sql('SectorDF3YLarge', engine, if_exists='replace')

In [18]:
StocksWide3Y = pd.read_sql_table('StocksWide3Y', engine)
SectorDF3Y = prep_data(StocksWide3Y, sectors, df500sectors)
SectorDF3YNegative = negative(SectorDF3Y)
SectorDF3YNegative['SPY'].value_counts()

c:\Users\canav\Documents\Fintech\machine_learning_index_prediction\utils\prep_data.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SectorDF[sector] = TempDF.mean(axis=1)


 1.0    129
-1.0    124
 0.0      1
Name: SPY, dtype: int64

In [20]:
SectorDF3YNegative.to_sql('SectorDF3YNegative', engine, if_exists='replace')

# 5 Day Moving Average

Work in process, do not reference past this

In [25]:
StocksWide = pd.read_sql_table('StocksWide', engine)
StocksWide.head()

,index,timestamp,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,...,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS,SPY Open,SPY Close
0,0,2021-04-14 04:00:00,-0.008490,-0.025272,0.001600,-0.022000,-0.003013,0.009064,0.002824,-0.006947,...,0.017936,0.018744,0.007477,0.005360,-0.003552,-0.005917,0.015529,-0.005670,412.86,411.45
1,1,2021-04-15 04:00:00,0.004598,-0.021653,0.006214,0.004781,0.008491,0.005066,0.012594,0.013658,...,-0.004716,-0.005371,0.001019,-0.007259,0.018743,0.005353,-0.011160,0.009272,413.79,415.87
2,2,2021-04-16 04:00:00,0.003954,-0.003169,0.008574,-0.003308,0.003907,0.001590,-0.026277,-0.000322,...,-0.010651,-0.000306,-0.005585,-0.004889,-0.004979,0.004952,-0.004079,0.008179,417.29,417.26
3,3,2021-04-19 04:00:00,-0.006549,-0.019563,-0.005424,0.010110,0.005928,-0.004167,0.003415,0.007524,...,-0.005634,0.000611,-0.006974,-0.005660,0.003548,-0.007537,-0.004802,0.003582,416.26,415.21
4,4,2021-04-20 04:00:00,-0.004060,-0.030938,-0.020657,-0.013926,0.008417,0.010059,-0.002388,-0.012020,...,-0.016717,0.001989,-0.005638,0.007258,0.007419,-0.011536,-0.050245,0.004704,413.93,412.17


In [ ]:
WideDF.drop(columns='index', inplace=True)
WideDF = WideDF.set_index('timestamp')
SectorDF = WideDF[['SPY', 'SPY Open', 'SPY Close']]

stockstodrop = []
StocksWideNoSPY = WideDF.drop(columns={'SPY','SPY Open','SPY Close'})
for sector in sectors:
    for (columnName, columnValues) in StocksWideNoSPY.iteritems():
        if sector != df500sectors.loc[columnName]['GICS Sector']:
            stockstodrop.append(columnName)
    TempDF = StocksWideNoSPY.drop(columns=stockstodrop)
    stockstodrop = []
    SectorDF[sector] = TempDF.mean(axis=1)
SectorDF

In [11]:
# SP500_Constituents_details = pd.read_csv(Path("resources/constituents_financials.csv"), index_col='Symbol')
# SP500_Constituents_details = SP500_Constituents_details.sort_values(by=['Market Cap'], ascending=False)
# SP500_Constituents_details.head()

In [18]:
# StockCount = 0
# Sleeve = 1
# StockMarketCap = [[],[],[],[],[],[]]

# for index, row in SP500_Constituents_details.iterrows():
#     if StockCount == Sleeve * 100:
#         Sleeve += 1
#     StockMarketCap[Sleeve-1].append(index)
#     StockCount += 1